In [1]:
from bs4 import BeautifulSoup
import urllib 
import requests
import os
import pandas as pd
import cv2
from PIL import Image, ImageSequence
import shutil
from urllib.request import Request, urlopen



In [4]:
def crowling_web(url):
    fp = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    source = urlopen(fp).read();
    soup = BeautifulSoup(source, 'html.parser')
    
    return soup

In [5]:
li_cate_big_block = soup.findAll('div', class_='col_block')

cate = list()
category_li = list()
for cb_block in li_cate_big_block:
    tmp=list()
#     tmp_name =list()
    if cb_block.find('h5'):
#         print(cb_block.find('h5').get_text())
        if not 'INNERWEAR 이너웨어'==cb_block.find('h5').get_text():
            tmp.append(cb_block.find('h5').get_text())
        
            for c in cb_block.findAll('li'):
#                 print(c['id'], c.get_text())
                tmp.append([c['id'],c.get_text()])
    
    if tmp and '이번주 추천 특집'!=cb_block.find('h5').get_text():
        cate.append(tmp)  
        
    if cb_block.find('h5'):
        if '이번주 추천 특집'==cb_block.find('h5').get_text():
                category_li.append(cate)
                cate=list()
    
    
#     print('-'*20)

In [21]:
data_li = list()
for i, category_ in enumerate(category_li[:2]):
    if i==0:
        cate_sex = 'FEMALE'
    elif i==1:
        cate_sex = 'MALE'
        
    for category in category_:
        cate_big = category[0]
        for cate_code in category[1:]:
            url = "https://store-kr.uniqlo.com/display/displayShop.lecs?storeNo=83&siteNo=50706&displayNo={}".format(cate_code[0])
            soup = crowling_web(url)

            li_product = soup.findAll('p', class_='tumb_img')

            for p in li_product:
                url_product = 'https://store-kr.uniqlo.com/'+p.find('a')['href']
                soup2 = crowling_web(url_product)

                li_img = soup2.findAll('ul', id='prodThumbImgs')[0].findAll('a')
                imageStr = './data_uniclo/{}/{}/{}'.format(cate_sex, cate_big.replace(' ','_'), cate_code[1])
                if not os.path.isdir(imageStr):
                    os.makedirs(imageStr)

                for j,img_l in enumerate(li_img):
                    if j%10==0:
                        print('processing : {}/{}'.format(j, len(li_img)))
                    imgURL = img_l['href']
                    prod_name = img_l.find('img')['alt']
            #         print(imgURL)
                    img_name = imgURL[imgURL.rindex('/')+1:]
#                     print(os.path.join(imageStr, img_name))
                    if not img_name.split('_')[1] in ['1','2','45','49','50']:
                        print(prod_name, img_name)
                        data_li.append([prod_name, img_name])
                        urllib.request.urlretrieve(imgURL, os.path.join(imageStr, img_name))


processing : 0/13
울트라라이트다운컴팩트베스트 440954_6_570.jpg
울트라라이트다운컴팩트베스트 440954_7_570.jpg
울트라라이트다운컴팩트베스트 440954_8_570.jpg
울트라라이트다운컴팩트베스트 440954_9_570.jpg
울트라라이트다운컴팩트베스트 440954_10_570.jpg
울트라라이트다운컴팩트베스트 440954_11_570.jpg
울트라라이트다운컴팩트베스트 440954_14_570.jpg
울트라라이트다운컴팩트베스트 440954_15_570.jpg
울트라라이트다운컴팩트베스트 440954_18_570.jpg
processing : 10/13
울트라라이트다운컴팩트베스트 440954_19_570.jpg
울트라라이트다운컴팩트베스트 440954_20_570.jpg
울트라라이트다운컴팩트베스트 440954_21_570.jpg
processing : 0/14
울트라라이트다운베스트 443198_6_570.jpg
울트라라이트다운베스트 443198_7_570.jpg
울트라라이트다운베스트 443198_8_570.jpg
울트라라이트다운베스트 443198_9_570.jpg
울트라라이트다운베스트 443198_10_570.jpg
울트라라이트다운베스트 443198_11_570.jpg
울트라라이트다운베스트 443198_13_570.jpg
울트라라이트다운베스트 443198_15_570.jpg
울트라라이트다운베스트 443198_18_570.jpg
processing : 10/14
울트라라이트다운베스트 443198_19_570.jpg
울트라라이트다운베스트 443198_20_570.jpg
울트라라이트다운베스트 443198_21_570.jpg
울트라라이트다운베스트 443198_22_570.jpg
processing : 0/17
TPJ울트라라이트다운롱베스트 445619_6_570.jpg
TPJ울트라라이트다운롱베스트 445619_7_570.jpg
TPJ울트라라이트다운롱베스트 445619_8_570.jpg
TPJ울트라라이트다운롱베스트 445619_9_570.jp

In [22]:
df = pd.DataFrame(data_li, columns=['product_name', 'file_name'])

In [23]:
df.to_csv('data_uniclo.csv', encoding='utf-8-sig', index=False)